In [ ]:
import sys
DIR = "/home/tim/Research/torch-points3d/"
sys.path.insert(0, DIR)

from torch_points3d.datasets.grasp_classification import acronymvid
from torch_points3d.models.grasp_classification import minkowski
from torch_points3d.metrics.model_checkpoint import ModelCheckpoint

import matplotlib.pyplot as plt
import imageio

In [ ]:
# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-04/17-03-22"
# ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-04/18-14-55"
ckpt_dir = "/home/tim/Research/torch-points3d/outputs/2021-08-04/23-53-23"
check_name="MinkUNet14A"

In [ ]:
from omegaconf import OmegaConf
with open("/home/tim/Research/torch-points3d/conf/data/grasp_classification/acronymvid.yaml") as f:
    params = OmegaConf.load(f)

In [ ]:
dataset = acronymvid.AcronymVidDataset(params)

In [ ]:
model_checkpoint = ModelCheckpoint(ckpt_dir, check_name, "test", resume=False)

model = model_checkpoint.create_model(dataset, weight_name="acc")

In [ ]:
import torch
# device = torch.device("cpu")
device = torch.device("cuda")

In [ ]:
for i in range(0, 20):
    data = dataset.test_dataset[0][i]
    model.set_input(data, device)
    model.forward()

In [ ]:
i = 45

from tqdm import tqdm
for i in range(0, 100, 20):
    data = dataset.test_dataset[0][i]
    model.set_input(data, device)
    model.forward()


    out = model.output.detach().numpy()
    out = (out.reshape(-1, 300, 300) > 0).astype('uint8') * 255

    for im in out:
        plt.imshow(im)
        plt.pause(0.01)

    name = check_name + "".join(ckpt_dir.split('/')[-2:]) + f"_{i}.gif"
    imageio.mimsave(f"figs/preds_{name}", out)

    # gtname = f"gt_{i}.gif"
    # imageio.mimsave(f"figs/{gtname}", data.y.detach().numpy().reshape(-1, 300, 300))